In [1]:
#pip install requests

In [2]:
import requests
import json
from tkinter import *

Configurações para Executar Modelos em Docker Local 

In [3]:
#IP do servidor local da aplicação no docker:
app_ip = "localhost"

#Porta do servidor de predição de Fraude especificada no docker:
fraude_porta = "10005"

#Porta do servidor de predição de Inadimplencia especificada no docker:
inadimplencia_porta = "10006"

## Parte 1 – Propensão de inadimplência (Modelo 'Fake')

In [4]:
# informe o cpf
cpf01 = "123456789"

In [5]:
url01 = "http://{}:{}/inadimplencia?cpf={}".format(app_ip, inadimplencia_porta,cpf01)

response01 = requests.request("POST",url01)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'Consulta Risco de Inadimplencia para o CPF': '123456789',
 'Resultado': 1,
 'Situacao': 'inadimplente',
 'Mensagem': 'Obrigado pela chamada de API'}

## Parte 2 – Propensão a Fraude (Modelo 'Fake')

In [8]:
# informe o cpf
cpf02 = "459489461"

In [9]:
url02 = "http://{}:{}/inadimplencia?cpf={}".format(app_ip,inadimplencia_porta,cpf02)
response02 = requests.request("POST",url02)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'Consulta Risco de Inadimplencia para o CPF': '459489461',
 'Resultado': 0,
 'Situacao': 'normal',
 'Mensagem': 'Obrigado pela chamada de API'}

## Parte 3 – Criação da Plataforma Cognitiva (Model Manager) na Azure.

In [1]:
#IP do Maquina Virtual na Azure:
server_ip = "40.76.106.228"

#Porta do servidor de predição na Azure:
server_port = "443"

url = "http://{}:{}/predict".format(server_ip,server_port)

CPF do cliente será submetido aos dois modelos para a análise de Risco de Fraude e Inadimplencia.

In [3]:
# informe o cpf
cpf = "123456789"

In [5]:
response001 = requests.request("POST", url + f"?model=modelo01&cpf={cpf}" )
resposta001 = json.loads(response001.text.encode('utf8').decode())
resposta001

{'Consulta Risco de Inadimplencia para o CPF': '1234567890',
 'Resultado': 0,
 'Situacao': 'normal',
 'Mensagem': 'Obrigado pela chamada de API'}

In [6]:
response002 = requests.request("POST", url + f"?model=modelo02&cpf={cpf}" )
resposta002 = json.loads(response002.text.encode('utf8').decode())
resposta002

{'Consulta Risco de Fraude para o CPF': '1234567890',
 'Resultado': 'Grupo de Clientes de Baixa Renda, Persona Estudante Universitario, Alto Risco',
 'Mensagem': 'Obrigado pela chamada de API'}

## Parte 4 – Demonstração de Análise de Crédito com FrontEnd 
## (OBS: Necessário Ambiente na Azure estar ativo!!!)

Basicamente a Análise consiste em uma consulta fictícia em uma base de dados onde o Sistema levará em consideração os seguintes critérios de acordo com o resultado dos modelos:

Consulta Inadimplencia: Verifica se o cliente (cpf) possui boa reputação no mercado quanto à Inadimplencia de acordo com os seguntes resultados: (0 - Situação Normal, 1 - Inadimplente)

Caso o sistema retorne 1 - Inadimplente, a Concessão de Crédito para o cliente é Negada. 
Caso o sistema retorne 0 - Normal, será feita uma consulta quanto a Risco de Fraude.

Consulta Risco de Fraude: Com base em uma base histórica fictícia, será feita uma análise quanto a Risco de Fraude de acordo com agrupamentos de clientes e tipo de persona podendo resultar em tres categorias de risco: (Alto Risco, Medio Risco e Baixo Risco)

Caso o sistema retorne 'Alto Risco' - A Concessão de Crédito para o cliente sera Negada. 
Caso o sistema retorne 'Medio Risco' - A Concessão de Crédito será direcionada para uma Avaliação de acordo com as Regras do Banco/Financeira. 
Caso o sistema retorne 'Baixo Risco' - A Concessão de Crédito para o cliente sera Aceita. 

In [9]:
def analise_cred():

    cpf = get_cpf()

    response001 = requests.request("POST", url + f"?model=modelo01&cpf={cpf}" )
    resposta001 = json.loads(response001.text.encode('utf8').decode())
    response002 = requests.request("POST", url + f"?model=modelo02&cpf={cpf}" )
    resposta002 = json.loads(response002.text.encode('utf8').decode())

    resp_cpf['text'] = f'''
        Consulta CPF: {cpf}'''    

    if (resposta001['Resultado']) == 1:
        texto_resposta['text'] = f'''
        Consessão de Crédito Negada!
        Motivo: Cliente com Alto Risco de Inadimplencia no Mercado.'''    
    else:
        teste = resposta002['Resultado']
        x = teste.split(", ")
        if x[2] == 'Alto Risco':
            texto_resposta['text'] = f'''
            Consessão de Crédito Negada!
            {resposta002['Resultado']}
            Motivo: Cliente com Alto Risco de Fraude no Mercado.'''
        elif x[2] == 'Medio Risco':    
            texto_resposta['text'] = f'''
            Consessão de Crédito em Análise!
            {resposta002['Resultado']}
            Motivo: Cliente sob Avaliação do Banco.'''
        elif x[2] == 'Baixo Risco':    
            texto_resposta['text'] = f'''
            Consessão de Crédito Aprovada!
            {resposta002['Resultado']}
            Motivo: Cliente com Baixo Risco de Inadimplencia e Fraude no Mercado.'''

def get_cpf():
    cpf = e_cpf.get()
    return cpf

janela = Tk()
janela.title("Analise de Credito")

texto = Label(janela, text="Favor informar CPF para realizar a Analise")
texto.grid(column=0, row=0, padx=5, pady=5)

e_cpf = Entry(janela, text='CPF',width=25, justify='left', font=('',15), highlightthickness=1,relief='solid' )
e_cpf.grid(column=0, row=1, padx=5, pady=5)

botao = Button(janela, text="OK", command=analise_cred)
botao.grid(column=0, row=2, padx=5, pady=5)

resp_cpf = Label(janela, text="")
resp_cpf.grid(column=0, row=3, padx=5, pady=5)

texto_resposta = Label(janela, text="")
texto_resposta.grid(column=0, row=4, padx=5, pady=5)

janela.mainloop()
